In [1]:
# # relative import ../build/fast_kinematics
# import sys
# sys.path.append('../../')
import numpy as np
import time
import torch
import fast_kinematics
# import pytorch_kinematics as pk

In [2]:
N = 4096
print(torch.__version__)

2.2.2+cu118


In [4]:
# fask kinematics model
model = fast_kinematics.FastKinematics("../kuka_iiwa.urdf", N, "lbr_iiwa_link_7")

Joint name: lbr_iiwa_joint_1
Joint type: 1
Joint rotation: 0i + 0j + 0k + 1
Joint position: 0i + 0j + 0.1575k + 0
Joint axis: 0 0 1
Joint name: lbr_iiwa_joint_2
Joint type: 1
Joint rotation: -7.3123e-14i + 0.707107j + 0.707107k + -7.3123e-14
Joint position: 0i + 0j + 0.2025k + 0
Joint axis: 0 0 1
Parent joint name: lbr_iiwa_joint_1
Joint name: lbr_iiwa_joint_3
Joint type: 1
Joint rotation: -7.3123e-14i + 0.707107j + 0.707107k + -7.3123e-14
Joint position: 0i + 0.2045j + 0k + 0
Joint axis: 0 0 1
Parent joint name: lbr_iiwa_joint_2
Joint name: lbr_iiwa_joint_4
Joint type: 1
Joint rotation: 0.707107i + 0j + 0k + 0.707107
Joint position: 0i + 0j + 0.2155k + 0
Joint axis: 0 0 1
Parent joint name: lbr_iiwa_joint_3
Joint name: lbr_iiwa_joint_5
Joint type: 1
Joint rotation: 7.3123e-14i + 0.707107j + 0.707107k + -7.3123e-14
Joint position: 0i + 0.1845j + 0k + 0
Joint axis: 0 0 1
Parent joint name: lbr_iiwa_joint_4
Joint name: lbr_iiwa_joint_6
Joint type: 1
Joint rotation: 0.707107i + 0j + 0k + 

In [9]:
# pk chain
chain = pk.build_serial_chain_from_urdf(open("../kuka_iiwa.urdf").read(), "lbr_iiwa_link_7")
d = "cuda"
dtype = torch.float32

chain = chain.to(dtype=dtype, device=d)

In [10]:
th = torch.rand(N, 7, dtype=dtype, device=d, requires_grad=True)
all_times = []
J = None
for _ in range(100):
  start_time = time.time()
  J = chain.jacobian(th)
  end_time = time.time()
  all_times.append(end_time - start_time)
print(f"Time taken for jacobian_mixed_frame: {np.mean(all_times)} with std: {np.std(all_times)}")

Time taken for jacobian_mixed_frame: 0.00854259729385376 with std: 0.0011613306822710263


In [6]:
# the output of fast_kinematics is column major 1d array, so we need to reshape it
# experimenting to make sure we got it right
flat_array = torch.arange(1,43)
print(flat_array.view(-1,7,6).permute(0,2,1))

tensor([[[ 1,  7, 13, 19, 25, 31, 37],
         [ 2,  8, 14, 20, 26, 32, 38],
         [ 3,  9, 15, 21, 27, 33, 39],
         [ 4, 10, 16, 22, 28, 34, 40],
         [ 5, 11, 17, 23, 29, 35, 41],
         [ 6, 12, 18, 24, 30, 36, 42]]])


In [11]:
# flatten th
th = th.view(N*7)
all_times = []
J2 = None
for _ in range(1000):
  start_time = time.time()
  J2 = model.jacobian_mixed_frame_pytorch(th).view(-1,7,6).permute(0,2,1)
  end_time = time.time()
  all_times.append(end_time - start_time)
print(f"Time taken for fk: {np.mean(all_times)} with std: {np.std(all_times)}")

Time taken for fk: 4.538774490356445e-06 with std: 1.5257827219971919e-05


In [12]:
assert torch.allclose(J, J2, atol=1e-5)

In [13]:
# test out pinv vs lstsq

b = torch.rand(6, 1, dtype=dtype, device=d, requires_grad=False)
x_pinv = torch.zeros(N, 7, dtype=dtype, device=d)
all_times = []
for i in range(100):
  start_time = time.time()
  # J2 is a collection of matrices but pinv seems to work on the entire batch
  J2_pinv = torch.pinverse(J2)
  if i == 0: x_pinv = torch.matmul(J2_pinv, b)
  end_time = time.time()
  all_times.append(end_time - start_time)
print(f"Time taken for pinv: {np.mean(all_times)} with std: {np.std(all_times)}")

# unit test correctness of pinv for an entire batch
first_pinv = J2[0].pinverse()
first_x = first_pinv @ b
assert torch.allclose(first_x, x_pinv[0], atol=1e-5)

Time taken for pinv: 0.03223166942596436 with std: 0.0038884625993035726


In [14]:
all_times = []
x_lstsq = None
new_b = b.view(1,6,1).repeat(N,1,1)  # repeat the same b for all N
for i in range(100):
  start_time = time.time()
  x_lstsq = torch.linalg.lstsq(J2, new_b).solution
  end_time = time.time()
  all_times.append(end_time - start_time)
print(f"Time taken for lstsq: {np.mean(all_times)} with std: {np.std(all_times)}")

Time taken for lstsq: 0.024238510131835936 with std: 0.0020009472923546044


In [10]:
# checking to make sure the results are the same
print(J.shape)
print(J2.shape)
assert torch.allclose(J, J2, atol=1e-5)

torch.Size([4096, 6, 7])
torch.Size([4096, 6, 7])


In [15]:
# just for fun numpy version still faster
qpos = np.random.rand(7*N)
qpos = qpos.astype(np.float32)

all_times = []
for _ in range(1000):
  start_time = time.time()
  poses = model.jacobian_mixed_frame(qpos)
  poses = poses.reshape(N,-1,6).transpose(0,2,1)
  end_time = time.time()
  all_times.append(end_time - start_time)
print(f"Time taken for jacobian_mixed_frame: {np.mean(all_times)} with std: {np.std(all_times)}")

Time taken for jacobian_mixed_frame: 0.00032803034782409666 with std: 0.0004883862096649905
